In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from gensim import models
from gensim import corpora
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Phrases
from gensim.models.fasttext import FastText
from gensim.models.wrappers.fasttext import FastText as FT_wrapper
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary
from nltk.tokenize import sent_tokenize, word_tokenize
from stop_words import get_stop_words
from random import shuffle
import time
import re
import pylab as pl
from ipywidgets import FloatProgress
from IPython import display
import matplotlib.pyplot as plt
%matplotlib notebook

In [25]:
class Searcher():
    def __init__(self, w2v=None):
        # load w2v modle
        if w2v is None:
            print("start loading w2v, this might take a while")
            self._w2v = KeyedVectors.load_word2vec_format("../w2v/model/wiki-news-300d-1M.vec")
        else:
            self._w2v = w2v
        
        # get and process database
        self.get_database()
        self.save_database()
        
    def get_database(self):
        # load the dataset : including only each company's name, url and summary
        self._database = pd.read_csv("../input/InvestData_2017-Nov-22_0101.csv").iloc[:, [1, 5, 6]]
        # 1: company name, 5: company website, 6: company manual desc
        raw_texts = []
        # preprocess all the text data and remove any row without any useful data, and segment each word
        drop_list = []
        for row in self._database.itertuples():
            if not type(row[1]) is str or (not type(row[2]) is str and not type(row[3]) is str):
                # check if the row has data
                drop_list.append(row[0])
            else:
                # process each website and replace web address with texts crawled
                url = row[2]
                texts = self.get_text_from_url_and_its_children(url)
                if not texts:
                    # if cannot access url, replace url with Nan
                    self._database.iloc[row[0], 1] = np.nan
                else:
                    # replace the url with the crawled texts
                    texts = '   '.join(texts)
                    self._database.iloc[row[0], 1] = texts
                    
                # process text data of both manually summarized or crawled data
                tmp_text = []
                for col in [2, 3]:
                    real_col = col - 1
                    text = row[col]
                    if type(text) is str:
                        text = self.word_tokenize_string(text)
                        self._database.iloc[row[0], real_col] = text
                        tmp_text.append(text)
                # merge texts of same company
                tmp_text = '    '.join(tmp_text)
                raw_texts.append(tmp_text)
                
        # drop all the rows that do not have essential data
        self._database.drop(drop_list, inplace=True)
        # create similarity col for similarity search use
        self._database = self._database.assign(similarity=np.zeros(len(self._database)))
        
        # use the raw_texts to generate tfidf model
        self._tfidf, self._dictionary = get_tfidf_and_dictionary(raw_texts)
        
    def save_database(self):
        self._database.to_csv('crawled_database.csv')
        print("database save successful")
    
    def update_similarity(self, input_text, col=1):
        # get input text vector
        input_text_vector = self._get_doc_vector(input_text)
        i = 0
        for row in database.itertuples():
            row_text_vector = self._get_doc_vector(row[col+1])
            similarity = input_text_vector.dot(row_text_vector)
            database.iloc[i, -1] = similarity
            i += 1
        self._database = self._database.sort_values(by='similarity', ascending=False)
        return database

    def get_doc_vector(self, text):
        tokens = list(self._dictionary.token2id)
        # convert any unknown word to known word
        new_text = []
        for word in text.split():
            if word in tokens:
                new_text.append(word)
            elif word in w2v: # replace the unknow word with the most similar word in tokens of dictionary
                new_text.append(self._w2v.most_similar_to_given(word_list=tokens, w1=word))

        # start to calculate vector using tfidf weighted word vector sum
        # get tfidf weight
        tokenized_text = [self._dictionary.doc2bow(new_text)]
        tfidf_text = self._tfidf[tokenized_text][0]
        # sum weighted word vectors
        sum_vector = self._w2v['happy'] * 0 # get the size of the word vector
        for word_id, weight in tfidf_text:
            word = self._dictionary[word_id]
            sum_vector += self._w2v[word] * weight
        sum_vector /= np.sqrt(sum_vector.dot(sum_vector)) # normalize the vector

        return sum_vector
    
    def word_tokenize_string(self, text):
        stop_words = get_stop_words('en') # get too frequent word
        text = text.replace('\r', ' ').replace('\n', ' ') # remove symbols
        text = re.sub(r"http\S+", "", text) # remove urls
        # remove any word that present too frequently or cannot be converted to word vector
        text = [word for sent in sent_tokenize(text.lower()) for word in word_tokenize(sent) \
                if not word in stop_words and word in self._w2v]
        return ' '.join(text)
    
    @staticmethod
    def get_tfidf_and_dictionary(texts):
        # get dictionary of texts
        texts = [text.split() for text in texts]
        dictionary = corpora.Dictionary(texts)

        # get tfidf ranking model
        tokenized_texts = [dictionary.doc2bow(text) for text in texts]
        tfidf = models.TfidfModel(tokenized_texts)

        return tfidf, dictionary
    
    def get_text_from_url_and_its_children(self, main_url):
        print("starting to crawl main url: ", main_url)
        # check validity of main_url
        resp = self.url_is_valid(main_url)
        if not resp:
            print("main_url is not valid")
            return False

        text_data = []
        print("\nstarting to crawl all its children")
        # grab all urls in this web page
        urls = [main_url]
        urls.extend(self.get_urls_from_url(main_url))
        print("\n\nthese are the children links we crawled")
        print(urls, "\n")
        # grab all texts in each urls
        # this set is used to store all visited urls, be advised, some url will redirect you to other places
        visited = set()
        for url in set(urls):
            resp = self.url_is_valid(url)
            if not resp:
                url = main_url + url
                resp = self.url_is_valid(url)
                if not self.url_is_valid(url):
                    print("url:", url, "invalid")
                    continue
            # double check if the url is visited
            if resp.url in visited: # shouldn't be unless the url is redirected
                print('url:', url, "has been visited")
                if resp.url != url: # meaning its redirected
                    print('the url is redirected, try https')
                    # try https
                    url = url[:4] + 's' + url[4:]
                    print(url)
                    resp = self.url_is_valid(url)
                    if resp:
                        if resp.url == url:
                            print('try succeeded')
                    else:
                        continue
            if resp.url in visited:
                print('url:', url, "has been visited")
                continue
            else:
                visited.add(resp.url)
            # check if url is the child or sibling of main_url
            if self.url_compare(main_url, resp.url) < 10: # to avoid http://www.
                print('\nurl:', resp.url, 'might be irrelevent to', main_url, 'quit visiting\n')
                continue
            for text in self.get_texts_from_resp(resp):
                text_data.append(text)
        return text_data
        
    @staticmethod
    def get_urls_from_url(main_url):
        resp = requests.get(main_url)
        soup = BeautifulSoup(resp.content, 'html.parser')
        urls = []
        links = soup.find_all('a')
        for url in links:
            try:
                url = url.attrs['href']
                if len(url) > 5:
                    urls.append(url)
            except:
                pass
        return urls
    
    @staticmethod
    def get_texts_from_resp(resp):
        # parse the web response
        soup = BeautifulSoup(resp.content, 'html.parser')
        # find and filter texts
        print("These are texts under", resp.url)
        texts = soup.find_all('p')
        print("number of items grabed are", len(texts))
        texts = [text for text in texts if len(text.text) > 100]
        print("number of items after filtering", len(texts))
        # output texts
        for text in texts:
            print(text.text)
            yield text.text
    
    @staticmethod
    def url_is_valid(url):
        try:
            resp = requests.get(url)
            assert resp.status_code == 200
            return resp
        except:
            return False

    @staticmethod
    def url_compare(url_target, url_income):
        n_same_letter = 0.0
        # delete all http or https 
        if url_target[4] == 's':
            url_target = url_target[5:]
        else:
            url_target = url_target[4:]
        if url_income[4] == 's':
            url_income = url_income[5:]
        else:
            url_income = url_income[4:]
        # check similarity
        min_len = min(len(url_target), len(url_income))
        for i in range(min_len-1):
            if url_target[i] == url_income[i]:
                n_same_letter += 1
            else:
                break
        return n_same_letter

In [26]:
w2v = KeyedVectors.load_word2vec_format("../w2v/model/wiki-news-300d-1M.vec")
# w2v = KeyedVectors.load("../w2v/model/fasttext_w2v_vector_64")

In [27]:
searcher = Searcher(w2v=w2v)
# there might be main_url + url is not valid, because url and main_url has overlaps, or main_url is not the root
# must provide root url, or 
# use overlaps to do intelligent main_url + url

starting to crawl main url:  http://www.aecfafrica.org/

starting to crawl all its children


these are the children links we crawled
['http://www.aecfafrica.org/', '/about-us/who-we-are', '/about-us/our-history', '/the-aecf-board', '/the-aecf-management', '/about-us/funding-partners', '/about-us/strategic-partners', '/about-us/work-with-us', '/portfolio/overview', '/portfolio/agribusiness', '/portfolio/renewable-energy', '/portfolio/aecf-connect', '/portfolio/competitions', '/impact', '/knowledge-hub', '/media-centre/blog', '/media-centre/news', '/media-centre/videos', '/contact-us', '/about-us/who-we-are', '/about-us/our-history', '/the-aecf-board', '/the-aecf-management', '/about-us/funding-partners', '/about-us/strategic-partners', '/about-us/work-with-us', '/portfolio/overview', '/portfolio/agribusiness', '/portfolio/renewable-energy', '/portfolio/aecf-connect', '/portfolio/competitions', '/impact', '/knowledge-hub', '/media-centre/blog', '/media-centre/news', '/media-centre/video

url: http://www.aecfafrica.org//about-us/who-we-are has been visited
the url is redirected, try https
https://www.aecfafrica.org//about-us/who-we-are
These are texts under https://www.aecfafrica.org/about-us/who-we-are
number of items grabed are 7
number of items after filtering 3
The Africa Enterprise Challenge Fund (AECF) is a US $304 million private sector fund that provides catalytic funding to enterprises in 24 countries across sub-Saharan Africa. The Fund supports innovative commercial businesses in the agribusiness, renewable energy and adaptation to climate change technology sectors with the aim of reducing rural poverty, promoting resilient rural communities and creating jobs through private sector development. The AECF invests in businesses that are seen as “risky” and that struggle to meet traditional risk-return standards for commercial investors and which offer significant impact on poverty alleviation by creating jobs and sustainable incomes.
Since 2008, the AECF has been

url: http://www.aecfafrica.org//contact-us has been visited
the url is redirected, try https
https://www.aecfafrica.org//contact-us
These are texts under https://www.aecfafrica.org/contact-us
number of items grabed are 4
number of items after filtering 0
url: http://www.aecfafrica.org//portfolio/agribusiness has been visited
the url is redirected, try https
https://www.aecfafrica.org//portfolio/agribusiness
These are texts under https://www.aecfafrica.org/portfolio/agribusiness
number of items grabed are 9
number of items after filtering 6
Agriculture and agribusiness accounts for 32 percent of GDP in Sub-Saharan Africa and employs 65 percent of the work force. It is likely to have a bigger impact on poverty reduction than in other sectors as it offers the most direct route of raising returns to poor people’s main assets, i.e. land and labour. The AECF’s work in the agriculture sector is the oldest and largest of its investments with a total of US$ 183 million committed across Africa. 

url: http://www.aecfafrica.org/https://www.linkedin.com/company/africa-enterprise-challenge-fund has been visited
the url is redirected, try https
https://www.aecfafrica.org/https://www.linkedin.com/company/africa-enterprise-challenge-fund
url: http://www.aecfafrica.org//about-us/funding-partners has been visited
the url is redirected, try https
https://www.aecfafrica.org//about-us/funding-partners
These are texts under https://www.aecfafrica.org/about-us/funding-partners
number of items grabed are 19
number of items after filtering 10
The AECF funding partners include the Governments of Australia, Canada, Denmark, the Netherlands, Sweden and the UK, as well as The Consultative Group to Assist the Poor (CGAP) and The International Fund for Agricultural Development (IFAD). Each funding partner has a focus area or geographical region that it wishes to impact upon, and the AECF provides the means to target those focus areas/regions, to award investees and monitor and evaluate their perfor

url: http://www.aecfafrica.org//portfolio/renewable-energy has been visited
the url is redirected, try https
https://www.aecfafrica.org//portfolio/renewable-energy
These are texts under https://www.aecfafrica.org/portfolio/renewable-energy
number of items grabed are 5
number of items after filtering 2
Renewable Energy and Adaptation to Climate Technologies (REACT) window is demonstrating how renewable energy technologies and businesses have the potential to reach Africa’s rural communities. Limited access to sources of energy, (especially in rural areas) continues to undermine the productivity of households and businesses, limit growth, experience high emissions and deteriorating living standards. Climate change has the potential to reduce agricultural yields, increase water scarcity and the frequency of extreme weather. REACT has shown that private sector innovation has the potential to reach people in ways that large-scale government investment in grid extension and climate adaptatio

...

These are texts under https://www.drkfoundation.org/news-post/data-quest-why-this-wonky-criminal-justice-group-is-pulling-in-the-big-bucks/
number of items grabed are 5
number of items after filtering 1
The world of crowded jails, tense courtrooms, and cops on the beat is a far cry from the air-conditioned spaces data wonks usually inhabit. But according to Measures for Justice (MFJ), better data is key to reforming America’s hyper-localized criminal justice system. Plenty of well-heeled funders are inclined to agree. In fact, some of the hottest names in philanthropy are flocking to support the organization, which was founded by lawyer and author Amy Bach in 2011.
These are texts under https://www.drkfoundation.org/apply-for-funding/
number of items grabed are 6
number of items after filtering 3
The Draper Richards Kaplan Foundation seeks to dramatically improve the lives of people and the world around us through innovative strategies, systems changing approaches, and disrupting techn

These are texts under https://www.drkfoundation.org/about/history/
number of items grabed are 7
number of items after filtering 4
William H. Draper, III, and Robin Richards Donohoe began investing together in the venture industry in 1994, concentrating on companies in India. Since then, they have invested in more than 100 early stage information technology investments in India and the United States.
They formed the Draper Richards Foundation in 2001 with a fund totaling $14 million, the majority of which was contributed by Bill and Robin, with contributions from family members. Bill and Robin’s successful venture capital partnership shaped the approach of the foundation and helped create the model we follow today.
In 2010,their foundation became the Draper Richards Kaplan Foundation when former vice chairman of Goldman Sachs and professor of leadership at Harvard Business School, Robert S. Kaplan, (currently President & CEO of the Federal Reserve Bank of Dallas) joined as co-chair. Tog


url: https://twitter.com/i/web/status/954431773022281728 might be irrelevent to https://www.drkfoundation.org/ quit visiting


url: https://twitter.com/search?f=tweets&q=%23UrgencyofNow&src=hash might be irrelevent to https://www.drkfoundation.org/ quit visiting

These are texts under https://www.drkfoundation.org/news-post/scaling-solutions-toward-shifting-systems/
number of items grabed are 5
number of items after filtering 1
In this new report, produced by Rockefeller Philanthropy Advisors in collaboration with the Skoll, Porticus, Ford and Draper Richards Kaplan Foundations, we examine how funders can work in more collaborative ways to place longer-term, adaptive and responsive resources to accelerate scalable solutions to the world’s most pressing problems.
url: https://www.drkfoundation.org//about/news has been visited
the url is redirected, try https
httpss://www.drkfoundation.org//about/news

url: https://twitter.com/FoodCorps might be irrelevent to https://www.drkfoundation.o

These are texts under https://www.drkfoundation.org/apply-for-funding/what-we-fund/
number of items grabed are 9
number of items after filtering 3
We look for exceptional entrepreneurs
We seek out entrepreneurs who exhibit characteristics of extraordinary leadership: vision, intelligence, influence, ambition, discretion and follow-through. Draper Richards Kaplan entrepreneurs have proven track records that demonstrate a full spectrum of competencies.
We look for potential to scale 
To affect meaningful change upon the major challenges of our time, we need big solutions to big problems. We support social enterprises — non profit, for profit and hybrid organizations — that can expand enough to directly benefit a large number of beneficiaries and impart enough momentum to influence broader systems that encumber progress. Scaled organizations act as models to other groups in the sector and have the clout to affect policy, public opinion, and economies.
We look for sustainable impact
We loo

NameError: name 'database' is not defined

In [11]:
def rescue_code(function):
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))

In [12]:
def get_database(w2v):
    # load the dataset and all the text data
    database = pd.read_csv("../input/InvestData_2017-Nov-22_0101.csv").iloc[:, [1, 6, 9]]
    raw_texts = []
    # preprocess all the text data and remove any row without any useful data, and segment each word
    drop_list = []
    for row in database.itertuples():
        if not type(row[1]) is str or (not type(row[2]) is str and not type(row[3]) is str):
            drop_list.append(row[0])
        else:
            for col in [2, 3]:
                real_col = col - 1
                text = row[col]
                if type(text) is str:
                    text = word_tokenize_string(text, w2v)
                    text = ' '.join(text)
                    database.iloc[row[0], real_col] = text
                    raw_texts.append(text)
    database.drop(drop_list, inplace=True)
    database = database.assign(similarity=np.zeros(len(database)))

    return database, raw_texts

def get_tfidf_and_dictionary(texts):
    # get dictionary of texts
    texts = [text.split() for text in texts]
    dictionary = corpora.Dictionary(texts)
    
    # get tfidf ranking model
    tokenized_texts = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(tokenized_texts)
    
    return tfidf, dictionary

def word_tokenize_string(text, w2v):
    stop_words = get_stop_words('en')
    text = text.replace('\r', ' ').replace('\n', ' ')
    text = re.sub(r"http\S+", "", text)
    text = [word for sent in sent_tokenize(text.lower()) for word in word_tokenize(sent) if not word in stop_words and word in w2v]
    return text

def update_similarity(w2v, dictionary, tfidf, input_text, database):
    # get input text vector
    input_text_vector = get_doc_vector(input_text, w2v, dictionary, tfidf)
    i = 0
    for row in database.itertuples():
        row_text_vector = get_doc_vector(row[2], w2v, dictionary, tfidf)
        similarity = input_text_vector.dot(row_text_vector)
        database.iloc[i, -1] = similarity
        i += 1
    database = database.sort_values(by='similarity', ascending=False)
    return database

def get_doc_vector(text, w2v, dictionary, tfidf):
    tokens = list(dictionary.token2id)
    # convert any unknown word to known word
    new_text = []
    for word in text.split():
        if word in tokens:
            new_text.append(word)
        elif word in w2v: # replace the unknow word with the most similar word in tokens of dictionary
            new_text.append(w2v.most_similar_to_given(word_list=tokens, w1=word))
    
    # start to calculate vector using tfidf weighted word vector sum
    # get tfidf weight
    tokenized_text = [dictionary.doc2bow(new_text)]
    tfidf_text = tfidf[tokenized_text][0]
    # sum weighted word vectors
    sum_vector = w2v['happy'] * 0 # get the size of the word vector
    for word_id, weight in tfidf_text:
        word = dictionary[word_id]
        sum_vector += w2v[word] * weight
    sum_vector /= np.sqrt(sum_vector.dot(sum_vector)) # normalize the vector
    
    return sum_vector

In [14]:
database, texts = get_database(w2v)
tfidf, dictionary = get_tfidf_and_dictionary(texts)

In [15]:
pd.read_csv("../input/InvestData_2017-Nov-22_0101.csv").iloc[:, :]

,No,Organisation,Type_1,Type_2,Region,Website,Description,Relevant_Products_and_Services,SectorsIndustries,Funding_details,Requirements__Eligibility,Examples,Application,Contact,Additional_Information1,Additional_Information2,Additional_Information3
0,0.0,filler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,The Africa Enterprise Challenge Fund (AECF),NaN,NaN,Africa,http://www.aecfafrica.org/,The AECF is an Africa-based challenge fund tha...,NaN,NaN,1) Challenge Fund model where funding is award...,"Vary from competition to competition, but gene...",M-Kopa\r,General Procedure\r\n\r\nBefore:\r\n1) Submit ...,"The AECF Limited,\r\n\r\nWest End Towers, Kanj...",NaN,NaN,NaN
2,2.0,Alliance for a green revolution in Africa (AGRA),NaN,NaN,Africa,http://agra.org/grants/,AGRA aims to invest in projects that can have ...,NaN,NaN,No much information provided - companies are u...,NaN,NaN,NaN,Nairobi\r\n\r\nTel: +254 (20) 3675 000 / +254 ...,NaN,NaN,NaN
3,3.0,Global Innovation Fund,NaN,NaN,Africa,http://www.globalinnovation.fund,A government-sponsored non-profit fund in Lon...,NaN,NaN,"Amount: $230,000 max for pilot project\r\n* Pi...",More details: http://www.globalinnovation.fund...,NaN,Accept applications all year round\r\n\r\n1) I...,http://www.globalinnovation.fund/contact-us,NaN,NaN,NaN
4,4.0,Acumen Fund,NaN,NaN,Africa,http://acumen.org/,A charity organisation providing funding for e...,NaN,NaN,"Investment capital in the range of $0.25M-$3M,...",Social responsibility and demonstrated sustain...,M-Kopa\r\n\r\nD.Light (Global company; african...,Accept applications all year round\r\n\r\nMore...,Do not accept contacts prior to submission,NaN,NaN,NaN
5,5.0,Bamboo Finance,NaN,NaN,Africa,http://www.bamboocp.com/,Bamboo Finance is a commercial private equity ...,NaN,NaN,NaN,The company provides essential services afford...,NaN,"Bamboo Finance Africa\r\n7th Floor, Purshottam...",NaN,NaN,NaN,NaN
6,6.0,African Development Fund,NaN,NaN,Africa,http://www.afdb.org/en/about-us/corporate-info...,The ADF contributes to the promotion of econom...,NaN,NaN,Areas of funding: \r\nThe African Development ...,NaN,NaN,East Africa Regional Resource Center (EARC)\r\...,NaN,NaN,NaN,NaN
7,7.0,Willow Impact,NaN,NaN,Africa,http://www.willowimpact.com/,An impact investment firm that manages and adv...,NaN,NaN,NaN,"Geographic areas: Eastern Africa, the Middle E...",NaN,http://www.willowimpact.com/contact-us/email-u...,NaN,NaN,NaN,NaN
8,8.0,Vista Ventures Social Impact Fund,NaN,NaN,Africa,http://www.vistaventures.com/,A California-based fund to provide education a...,NaN,NaN,Seek early stage companies that wish to raise ...,"By early stage, we expect that applicant compa...",NaN,Submit:\r\n1) Business plan (or equivalent)\r\...,NaN,NaN,NaN,NaN
9,9.0,Grayghost ventures,NaN,NaN,Africa,http://www.grayghostventures.com/indexa.html,Seeks to eliminate poverty and strengthen comm...,NaN,NaN,Stage: Early-stage venture capital \r\n\r\nLoc...,Our investment approach prioritizes social ven...,M-Kopa,Submit an executive summary to\r\ninfo@graygho...,Headquarters:\r\nGray Ghost Ventures\r\n2200 C...,NaN,NaN,NaN


In [16]:
input_text = "new start up aiming at low income customers, dedicated in green energy"

In [17]:
search_output = update_similarity(w2v, dictionary, tfidf, input_text, database)
list(search_output.iloc[1, :])

['Fund for Developing',
 '. invests enterprises low income countries promote business development contribute economic growth poverty alleviation . ’ s geographic focus eastern southern africa , well selected countries asia central america . focuses supporting small medium sized companies .',
 nan,
 0.87590879201889038]

In [44]:
vec1 = get_doc_vector('a startup that dedicate to green energy', w2v, dictionary, tfidf)
vec2 = get_doc_vector('business regrading green energy', w2v, dictionary, tfidf)
vec3 = get_doc_vector('companies specificly support low-income people', w2v, dictionary, tfidf)

In [46]:
vec1.dot(vec2)

0.83994347

In [11]:
vec1

array([-0.01663876, -0.00337446, -0.02395986, -0.04154515, -0.10667257,
        0.01662513, -0.01009543,  0.06161033,  0.05003018, -0.06181473,
       -0.02179295, -0.01145902, -0.01362746, -0.03405415,  0.00218448,
        0.00767476, -0.01452028, -0.0018257 ,  0.07659619, -0.04767921,
       -0.04837132, -0.02310556, -0.05546847, -0.00884663, -0.01575219,
        0.00686825, -0.02110741,  0.02846674, -0.00410359,  0.0267566 ,
        0.0219034 ,  0.01259031, -0.00153076,  0.0267643 ,  0.04200932,
       -0.05939888,  0.02810077, -0.01088257,  0.02167377,  0.05022477,
        0.00617732, -0.01705034,  0.04542316,  0.08962091, -0.01164754,
        0.0392498 , -0.02558717, -0.0163672 ,  0.06803226, -0.01166763,
        0.01418332,  0.04633227, -0.00154001,  0.00630221, -0.05789408,
        0.02684514,  0.03213207,  0.02908244, -0.05910822,  0.01098826,
       -0.04702628, -0.01257465,  0.02256784, -0.01471028], dtype=float32)

In [12]:
vec2

array([-0.01663876, -0.00337446, -0.02395986, -0.04154515, -0.10667257,
        0.01662513, -0.01009543,  0.06161033,  0.05003018, -0.06181473,
       -0.02179295, -0.01145902, -0.01362746, -0.03405415,  0.00218448,
        0.00767476, -0.01452028, -0.0018257 ,  0.07659619, -0.04767921,
       -0.04837132, -0.02310556, -0.05546847, -0.00884663, -0.01575219,
        0.00686825, -0.02110741,  0.02846674, -0.00410359,  0.0267566 ,
        0.0219034 ,  0.01259031, -0.00153076,  0.0267643 ,  0.04200932,
       -0.05939888,  0.02810077, -0.01088257,  0.02167377,  0.05022477,
        0.00617732, -0.01705034,  0.04542316,  0.08962091, -0.01164754,
        0.0392498 , -0.02558717, -0.0163672 ,  0.06803226, -0.01166763,
        0.01418332,  0.04633227, -0.00154001,  0.00630221, -0.05789408,
        0.02684514,  0.03213207,  0.02908244, -0.05910822,  0.01098826,
       -0.04702628, -0.01257465,  0.02256784, -0.01471028], dtype=float32)